#  Keras

Author: __Thanchanok Phuawiriyakul 24582239__

        AT3 - Data Product with Machine 
In this experiment, we will train the model using Keras to predict the total fares.


__Table of contents__
1. [Load dataset](#load-dataset)
2. [Train Keras](#Train-keras )
3. [Keras tuner](#Keras-Tuner)
4. [Conclusion]()
5. [Reference]()

In [13]:
# first neural network with keras tutorial
import numpy as np
import tensorflow as tf
import keras
import sys
import pandas as pd
import keras_tuner
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend


In [14]:
sys.path.append('../../src')

## Load dataset

In [15]:
from data.make_dataset import *
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../../data/processed/')

## Train keras

In [37]:
# define the keras model
model = keras.Sequential()
model.add(Dense(units=12, input_shape=(12,), activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [38]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [39]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
253500/253500 [==============================] - 97s 381us/step - loss: 0.5122
Epoch 2/5
253500/253500 [==============================] - 95s 375us/step - loss: 0.5011
Epoch 3/5
253500/253500 [==============================] - 100s 395us/step - loss: 0.4983
Epoch 4/5
253500/253500 [==============================] - 99s 392us/step - loss: 0.4947
Epoch 5/5
253500/253500 [==============================] - 98s 386us/step - loss: 0.4935


In [40]:
loss = model.evaluate(X_train, y_train)
print(f"MSE test set: {loss}")

253500/253500 [==============================] - 90s 354us/step - loss: 0.4943
MSE test set: 0.49432602524757385


In [20]:
loss = model.evaluate(X_test, y_test)
print(f"MSE test set: {loss}")

84500/84500 [==============================] - 35s 415us/step - loss: 0.5040
MSE test set: 0.5039887428283691


In [43]:
y_pred = model.predict(X_test)


84500/84500 [==============================] - 28s 327us/step


In [41]:
model.save('../../models/keras.keras')

## Keras Tuner

In [43]:
# pip install keras-tuner --upgrade

Note: you may need to restart the kernel to use updated packages.


In [44]:
def build_model_tune(hp):
  model_tune = Sequential()
  model_tune.add(Dense(hp.Choice('units', [8, 16, 32]), input_shape=(12,), activation='relu'))
  model_tune.add(Dense(1, activation='linear'))

  model_tune.compile(loss='mean_squared_error', optimizer='adam')
  return model_tune

In [45]:
tuner = keras_tuner.RandomSearch(
    build_model_tune,
    objective='val_loss',
    max_trials=5)

In [46]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
best_model = tuner.get_best_models()[0]

Trial 3 Complete [00h 12m 22s]
val_loss: 0.4709257483482361

Best val_loss So Far: 0.4709257483482361
Total elapsed time: 00h 33m 15s


In [47]:
loss = best_model.evaluate(X_train, y_train)
print(f"MSE test set: {loss}")

253500/253500 [==============================] - 88s 347us/step - loss: 0.4721
MSE test set: 0.47212257981300354


In [48]:
loss = best_model.evaluate(X_test, y_test)
print(f"MSE test set: {loss}")

84500/84500 [==============================] - 33s 396us/step - loss: 0.4710
MSE test set: 0.4709739685058594


In [50]:
best_model.save('../../models/keras_tuner.keras')

## Save model

In [51]:
model = keras.models.load_model('../../models/keras_tuner.keras')

## test

In [8]:
from datetime import datetime, time
X = pd.DataFrame({
        'flightDate': '2023-11-07',
        'startingAirport':15,
        'destinationAirport':10,
        'departureTime': time(10, 30), 
        'segmentsCabinCode':0,
        'isNonStop': 1, 
        'isBasicEconomy': [0],  # Set default value to False
        'totalTravelDistance': [1569.618] #Mean total travel distance
        })
    
# Transform date column: flightDate
X['flightDate'] = pd.to_datetime(X['flightDate'])
X['flightDate_day'] = X['flightDate'].dt.day
X['flightDate_month'] = X['flightDate'].dt.month
X['flightDate_year'] = X['flightDate'].dt.year

# hour = departure_time.hour
# minute = departure_time.minute
X['DepartTime_hour'] = 10   
X['DepartTime_minute'] = 30
X['DepartTime_second'] = 0  # Since seconds are always 00

# Drop date columns
X = X.drop(columns=['flightDate','departureTime'])
X = X[['totalTravelDistance', 'isNonStop', 'isBasicEconomy', 'startingAirport', 'destinationAirport', 'segmentsCabinCode','flightDate_day', 'flightDate_month', 'flightDate_year',
                    'DepartTime_hour', 'DepartTime_minute', 'DepartTime_second']]
X

,totalTravelDistance,isNonStop,isBasicEconomy,startingAirport,destinationAirport,segmentsCabinCode,flightDate_day,flightDate_month,flightDate_year,DepartTime_hour,DepartTime_minute,DepartTime_second
0,1569.618,1,0,15,10,0,7,11,2023,10,30,0


In [55]:
model2 = keras.models.load_model('../../models/keras.keras')

In [52]:
y = model.predict(X)

1/1 [==============================] - 0s 30ms/step


In [53]:
y

array([[4359.4023]], dtype=float32)

In [56]:
y = model2.predict(X)

1/1 [==============================] - 0s 33ms/step


In [57]:
y

array([[323.83466]], dtype=float32)